In [1]:
# Load TensorBoard notebook extension
%load_ext tensorboard

Run ``tfds build --overwrite`` under lfw_pairs directory first.

# Imports

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt

from deepface import DeepFace

2024-01-30 18:23:36.636488: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-30 18:23:36.673380: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 18:23:36.673403: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 18:23:36.674346: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 18:23:36.680598: I tensorflow/core/platform/cpu_feature_guar

In [3]:
LOG_DIR = './logs/' + datetime.now().strftime("%Y%m%d-%H%M%S")
FILE_WRITER = tf.summary.create_file_writer(LOG_DIR)

2024-01-30 18:23:39.482630: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-30 18:23:39.518613: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-30 18:23:39.520047: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

# Dataset

In [4]:
ds, info = tfds.load('lfw_pairs',
                      with_info=True,
                      download=True,
                      as_supervised=False)

In [5]:
bench_ds = ds['train'].batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
tfds.benchmark(bench_ds, batch_size=32)


************ Summary ************



  0%|          | 0/69 [00:00<?, ?it/s]

Examples/sec (First included) 5742.08 ex/sec (total: 2240 ex, 0.39 sec)
Examples/sec (First only) 499.06 ex/sec (total: 32 ex, 0.06 sec)
Examples/sec (First excluded) 6773.37 ex/sec (total: 2208 ex, 0.33 sec)


,duration,num_examples,avg
first+lasts,0.390102,2240,5742.080725
first,0.064120,32,499.064332
lasts,0.325983,2208,6773.369695


In [6]:
with FILE_WRITER.as_default():
    tmp = np.array([x['img_0'] for x in ds['train'].take(25).as_numpy_iterator()])
    tf.summary.image("Pair 0", tmp, max_outputs=25, step=0)
    tmp = np.array([x['img_1'] for x in ds['train'].take(25).as_numpy_iterator()])
    tf.summary.image("Pair 1", tmp, max_outputs=25, step=0)

2024-01-30 18:23:40.400472: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [7]:
info.features

FeaturesDict({
    'img_0': Image(shape=(250, 250, 3), dtype=uint8),
    'img_1': Image(shape=(250, 250, 3), dtype=uint8),
    'pair': ClassLabel(shape=(), dtype=int64, num_classes=2),
})

In [8]:
info

tfds.core.DatasetInfo(
    name='lfw_pairs',
    full_name='lfw_pairs/0.1.0',
    description="""
    TODO(lfw_pairs): Markdown description of that will appear on the catalog page.
    Description is **formatted** as markdown.
    
    It should also contain any processing which has been applied (if any),
    (e.g. corrupted example skipped, images cropped,...):
    """,
    homepage='http://vis-www.cs.umass.edu/lfw',
    data_dir='/home/coder/tensorflow_datasets/lfw_pairs/0.1.0',
    file_format=tfrecord,
    download_size=172.28 MiB,
    dataset_size=87.63 MiB,
    features=FeaturesDict({
        'img_0': Image(shape=(250, 250, 3), dtype=uint8),
        'img_1': Image(shape=(250, 250, 3), dtype=uint8),
        'pair': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=None,
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=1000, num_shards=1>,
        'train': <SplitInfo num_examples=2200, num_shards=1>,
    },
    citation="

# Using DeepFace

In [9]:
for x in ds['test'].take(1).as_numpy_iterator():
  print(DeepFace.represent(x['img_0']))

2024-01-30 18:23:40.900923: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-01-30 18:23:42.015540: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-30 18:23:42.081900: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


[{'embedding': [0.1142425975272776, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.017759845743876138, 0.0, 0.0, 0.0, 0.02476352951549908, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.036325212874510515, 0.0, 0.0, 0.048383725601585076, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.060922384244081716, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.029414115423410724, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0020794958286390543, 0.0, 0.0, 0.0, 0.0, 0.07078770204444165, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002124698761759256, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06251787370163364, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0544030058483168, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04716869184168434, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [10]:
model = DeepFace.build_model(model_name='Facenet')

In [11]:
gt = []
pred = []

models = [
  "VGG-Face", 
  "Facenet", 
  "Facenet512", 
  "OpenFace", 
  "DeepFace", 
  "DeepID", 
  "ArcFace", 
  "Dlib", 
  "SFace",
]

ds_test = ds['test'].batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

for x in ds['train'].take(50).as_numpy_iterator():
  pred.append(DeepFace.verify(x['img_0'], x['img_1'], model_name=models[3], enforce_detection=False)['verified'])
  gt.append(x['pair'])

In [12]:
from sklearn.metrics import accuracy_score
print(accuracy_score(gt, pred))

0.54
